In [10]:
import pandas as pd
import matplotlib.pyplot as plt
import pulp
import sys
sys.path.append('../modules/')
from mymodule import *
from pulp_func import *

df2023_result = pd.read_excel('../excels/附件3/2023_result.xlsx')
# 去除列名末尾的空格。很奇怪
df2023_result.columns = [column.rstrip(' ') for column in df2023_result.columns]
# print(df2023_result)

In [11]:
result1 = df21.groupby(['作物编号', '地块类型', '种植季次'])['种植面积/亩'].sum().reset_index()
result1.columns = ['作物编号', '地块类型', '种植季次', '种植面积/亩']
merged_df = pd.merge(result1, df22[['作物编号', '作物名称','作物类型' , '地块类型', '种植季次', '亩产量/斤', '种植成本/(元/亩)', '销售单价/(元/斤)']], on=['作物编号', '种植季次', '地块类型'], how='left')
merged_df['单地块类型总产量/斤'] = merged_df['种植面积/亩'] * merged_df['亩产量/斤']
merged_df['销售单价平均值'] = merged_df['销售单价/(元/斤)'].apply(calculate_average_price)
merged_df['利润'] = (merged_df['销售单价平均值'] * merged_df['单地块类型总产量/斤']) - (merged_df['种植成本/(元/亩)'] * merged_df['种植面积/亩'])
merged_df['每亩利润'] = merged_df['利润'] / merged_df['种植面积/亩']
# print(merged_df)
total_profit_2023 = merged_df.groupby(['作物编号', '作物名称'])[['利润', '每亩利润']].sum().reset_index()
total_profit_2023.columns = ['作物编号', '作物名称', '总利润/元', '每亩总利润/元']
# print(total_profit)
total_yield_2023 = merged_df.groupby(['作物编号', '作物名称'])['单地块类型总产量/斤'].sum().reset_index()
total_yield_2023.columns = ['作物编号', '作物名称', '总产量/斤']
# print(total_yield_2023)

In [12]:
fields = create_fields(df11)
# for field in fields:
#     print(field)
new_fields = create_new_fields(fields)
# for field in new_fields:
#     print(field)
crops = create_crops(df12, df22)
# for crop in crops:
#     print(crop)    

In [13]:
# 示例调用

fieldtest = new_fields[-0]
croptest = crops[0]
print(fieldtest.field_name, fieldtest.season, croptest.crop_name)
yield_value = get_yield(fieldtest, croptest)
cost_value = get_cost(fieldtest, croptest)
print(f"Yield for {croptest.crop_name} in {fieldtest.field_name}: {yield_value}")
print(f"Cost for {croptest.crop_name} in {fieldtest.field_name}: {cost_value}")
total_yield = get_expected_sales(croptest, total_yield_2023)
print(f"Total yield for {croptest.crop_name} in 2023: {total_yield}")
print(f"{fieldtest.field_name} {croptest.crop_name} {check_crop_constraints(fieldtest, croptest)}")
print(check_rotation_constraints(new_fields[0], crops[0], df2023_result)) # 黄豆2023年第一季在A1地块未种植
print(check_rotation_constraints(new_fields[0], crops[5], df2023_result)) # 小麦2023年第一季在A1地块种植

A1 单季 黄豆
Yield for 黄豆 in A1: 400
Cost for 黄豆 in A1: 400
Total yield for 黄豆 in 2023: 57000.0
A1 黄豆 True
True
False


In [14]:
variables, binary_variables, actual_sales, excess_yield = create_variables(new_fields, crops)
# 创建线性规划问题
prob = pulp.LpProblem("Crop_Planting_Optimization", pulp.LpMaximize)

In [15]:
# 定义目标函数的各个部分

# 收入部分：实际销售量 * 作物价格
revenue = pulp.lpSum([
    actual_sales[crop.crop_name] * crop.crop_price
    for crop in crops
])

# 成本部分：种植成本
cost = pulp.lpSum([
    variables[(field.field_name, crop.crop_name, field.season)] * get_cost(field, crop)
    for field in new_fields for crop in crops
])

# 超额部分处理：超额部分 * 作物价格 * 系数
k = 0  # 系数
excess_handling = pulp.lpSum([
    excess_yield[crop.crop_name] * crop.crop_price * k
    for crop in crops
])

# 总利润 = 收入 - 成本 + 超额部分处理
profit = revenue - cost + excess_handling

# 将目标函数添加到问题中
prob += profit

# print(profit)

In [16]:
# 添加约束条件
# 地块面积限制
for field in new_fields:
    prob += pulp.lpSum([variables[(field.field_name, crop.crop_name, field.season)] for crop in crops]) <= field.field_area

# 作物种植限制
for field in new_fields:
    for crop in crops:
        if not check_crop_constraints(field, crop):
            prob += variables[(field.field_name, crop.crop_name, field.season)] == 0

# 重茬限制
for field in new_fields:
    for crop in crops:
        if not check_rotation_constraints(field, crop, df2023_result):
            prob += variables[(field.field_name, crop.crop_name, field.season)] == 0

# 豆类作物种植要求
# for field in new_fields:
#     prob += pulp.lpSum([variables[(field.field_name, crop.crop_name, field.season)] 
#                         for crop in crops if crop.crop_type in ['粮食（豆类）', '蔬菜（豆类）']]) >= 1

# 实际销售量约束
for crop in crops:
    actual_yield = pulp.lpSum([
        variables[(field.field_name, crop.crop_name, field.season)] * get_yield(field, crop)
        for field in new_fields
    ])
    expected_sales = get_expected_sales(crop, total_yield_2023)
    prob += actual_sales[crop.crop_name] <= actual_yield
    prob += actual_sales[crop.crop_name] <= expected_sales

    # 超额部分约束
    prob += excess_yield[crop.crop_name] == actual_yield - expected_sales # 这里不是>=，应该是==？！
    prob += excess_yield[crop.crop_name] >= 0

# # 作物在单个地块某一季节种植面积不宜太小
# min_area_percent = 0.2
# for field in new_fields:
#     for crop in crops:
#         var = variables[(field.field_name, crop.crop_name, field.season)]
#         binary_var = binary_variables[(field.field_name, crop.crop_name, field.season)]
#         min_area = min_area_percent * field.field_area
#         prob += var >= min_area * binary_var
#         prob += var <= field.field_area * binary_var

# 作物种植不宜太分散
max_plots = 4
for crop in crops:
    for field_type in set(field.field_type for field in new_fields):
        if field_type not in ['普通大棚', '智慧大棚']:
            prob += pulp.lpSum([
                binary_variables[(field.field_name, crop.crop_name, field.season)]
                for field in new_fields if field.field_type == field_type
            ]) <= max_plots

In [17]:
# 求解问题
prob.solve()
print("Status:", pulp.LpStatus[prob.status])
# print("Objective:", pulp.value(prob.objective))

# 打印所有变量及其值
# for v in prob.variables():
#     print(v.name, "=", v.varValue)

# for v in prob.variables():
#     if v.varValue > 0:
#         print(v.name, "=", v.varValue)

# 打印所有约束条件
# for name, constraint in prob.constraints.items():
#     print(f"{name}: {constraint}")

# 打印模型的详细信息
# print(prob)

Status: Optimal


In [20]:
update_new_fields(variables, new_fields)
# 打印结果
# for field in new_fields:
#     print(field.field_name, field.season, field.planted_crop)
df_template = pd.read_excel('../excels/附件3/template.xlsx')
output(df_template, 2024, new_fields, k)
# print(df_template)

e:\Users\Timothy\anaconda3\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
